In [ ]:
from acr.utils import SOM_BLUE, ACR_BLUE, NNXR_GRAY

MAIN_EXP = 'swi'
SUBJECT_TYPE = 'acr'
MAIN_COLOR = ACR_BLUE

In [ ]:

#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import pingouin as pg
from scipy.stats import shapiro, normaltest
import numpy as np
import kdephys as kde

from cmcrameri import cm as scm


import warnings
warnings.filterwarnings('ignore')

probe_ord = ['NNXr', 'NNXo']
hue_ord = [NNXR_GRAY, MAIN_COLOR]

from acr.plots import lrg
plt.rcdefaults()
lrg()
plt.rcParams['xtick.bottom'] = False
#--------------------------------- Import Publication Functions ---------------------------------#
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from acr.utils import *

In [ ]:
%connect_info

In [ ]:
notebook_figure_root = f'{pu.PAPER_FIGURE_ROOT}/synchrony/sttc'

In [ ]:
subjects, exps = pu.get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
subjects, exps = pu.remove_subject('ACR_16', subjects, exps)

In [ ]:
muas = {}
for subject, exp in zip(subjects, exps):
    mua = acr.mua.load_concat_peaks_df(subject, exp)
    muas[subject] = mua

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True, duration='3600s')

In [ ]:
bmtx = {}
for subject in muas.keys():
    bmtx[subject] = {}
    for condition in ['early_bl', 'circ_bl', 'stim', 'rebound']:
        hypno_to_use = hyp_dicts[subject][condition]
        bmtx[subject][condition] = acr.sync.dual_probe_sttc(muas[subject], hypno_to_use, delta_ms=5)
for subject in muas.keys():
    hypno_to_use = acr.hypnogram_utils.get_full_bl_hypno(full_hyps[subject], state='NREM')
    bmtx[subject]['full_bl'] = acr.sync.dual_probe_sttc(muas[subject], hypno_to_use, delta_ms=5)

In [ ]:
nan_out_chans = False
# nan out obviously artifactual channels
scd = {}
scd['ACR_17'] = [12, 13]
scd['ACR_31'] = [0]

conds = ['rebound', 'full_bl', 'early_bl', 'circ_bl', 'stim']
if nan_out_chans:
    for subject in scd.keys():
        for probe in ['NNXo', 'NNXr']:
            for condition in conds:
                old_dat = bmtx[subject][condition][probe]
                new_dat = acr.sync.mask_bad_channels(old_dat, scd[subject])
                bmtx[subject][condition][probe] = new_dat

In [ ]:
mtx_avgs = []
for subject in bmtx.keys():
    for cond in bmtx[subject].keys():
        for probe in bmtx[subject][cond].keys():
            for i, bout_mat in enumerate(bmtx[subject][cond][probe]):
                avg = np.nanmean(bout_mat)
                mtx_avg = pd.DataFrame(
                    {
                        'condition': cond,
                        'bout_ix': i,
                        'avg': avg,
                        'probe': probe,
                        'subject': subject
                    },
                    index=[0])
                mtx_avgs.append(mtx_avg)
mtxdf = pd.concat(mtx_avgs)
mtxdf = pl.from_dataframe(mtxdf)
mtxdf = dag.relativize_df(mtxdf, 'condition', 'full_bl', 'mean', 'avg', ['subject', 'probe'])
mtxmean = mtxdf.cdn('rebound').group_by(['subject', 'probe']).agg(pl.col('avg_rel').mean())
mtxmean = mtxmean.sort(['subject', 'probe'])

In [ ]:
# average across bouts (i.e. keep channel-pair info) for all conditions
cond_avgs = {}
for subject in bmtx.keys():
    cond_avgs[subject] = {}
    for cond in bmtx[subject].keys():
        cond_avgs[subject][cond] = {}
        for probe in bmtx[subject][cond].keys():
            cond_avgs[subject][cond][probe] = acr.sync.average_sttc_matrices(bmtx[subject][cond][probe])

In [ ]:
# make the averaged condition-matrices all relative to the full baseline
reb_rel_avgs = {}
for subject in bmtx.keys():
    reb_rel_avgs[subject] = {}
    for probe in ['NNXr', 'NNXo']:
        full_bl_mtx = cond_avgs[subject]['full_bl'][probe]
        reb_mtx = cond_avgs[subject]['rebound'][probe]
        rel_mtx = reb_mtx / full_bl_mtx
        reb_rel_avgs[subject][probe] = rel_mtx

In [ ]:
# make the averaged condition-matrices all relative to the full baseline - FISHER DIFFERENCE
reb_rel_avgs = {}
for subject in bmtx.keys():
    reb_rel_avgs[subject] = {}
    for probe in ['NNXr', 'NNXo']:
        full_bl_mtx = cond_avgs[subject]['full_bl'][probe]
        reb_mtx = cond_avgs[subject]['rebound'][probe]
        eps = 1e-6
        full_bl_mtx = np.arctanh(np.clip(full_bl_mtx, -1+eps, 1-eps))
        reb_mtx = np.arctanh(np.clip(reb_mtx, -1+eps, 1-eps))
        rel_mtx = reb_mtx - full_bl_mtx
        reb_rel_avgs[subject][probe] = rel_mtx

In [ ]:
#obvious outlier/artifactual pairs
reb_rel_avgs['ACR_18']['NNXo'][1, 2] = np.nan
reb_rel_avgs['ACR_18']['NNXr'][1, 2] = np.nan
reb_rel_avgs['ACR_18']['NNXo'][3, 4] = np.nan
reb_rel_avgs['ACR_18']['NNXr'][3, 4] = np.nan
reb_rel_avgs['ACR_18']['NNXo'][4, 5] = np.nan
reb_rel_avgs['ACR_18']['NNXr'][4, 5] = np.nan

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False

In [ ]:
# better averaging method - first average each condition across bouts into a single matrix, then make each channel-pair value relative to its own channel-pair value from the baseline. Then average all of those channel-pairs into a single number.
fig_id = 'REBOUND_STTC_all_channel_average'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxr = np.array([np.nanmean(reb_rel_avgs[subject]['NNXr']) for subject in reb_rel_avgs.keys()])
nnxo = np.array([np.nanmean(reb_rel_avgs[subject]['NNXo']) for subject in reb_rel_avgs.keys()])
nnxr = nnxr[~np.isnan(nnxr)]
nnxo = nnxo[~np.isnan(nnxo)]

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=(3.5, 4))
print(ax.get_ylim())
ax.set_yticks([0, .025, .05 ])
ax.set_ylim(-0.01, 0.05)

f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
#stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))

if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
nnxr_norm = nnxr-nnxr
nnxo_norm = nnxo-nnxr

# better averaging method - first average each condition across bouts into a single matrix, then make each channel-pair value relative to its own channel-pair value from the baseline. Then average all of those channel-pairs into a single number.
fig_id = 'REBOUND_STTC_all_channel_average-NORMALIZED'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = acr.plots.gen_paired_boxplot(nnxr_norm, nnxo_norm, colors=[NNXR_GRAY, MAIN_COLOR], fsize=(3.5, 4), one_sided=True)
#ax.set_yticks([0.8, 0.9, 1])
#ax.set_ylim(0.74, 1.01)

f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# Matrix - Main Plots 

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib.colors import ListedColormap
from matplotlib.colors import TwoSlopeNorm

def modified_coolwarm_low(low="#1a9850", *, N=256, name="coolgreen"):
    """
    Return a version of 'coolwarm' where the *lower* half fades
    into `low` (instead of blue).

    Parameters
    ----------
    low : str or tuple
        Target low-end colour (e.g. "#1a9850" for green).
    N : int
        Number of discrete samples pulled from the base map (256 by default).
    name : str
        Name given to the resulting colormap object.

    Returns
    -------
    matplotlib.colors.ListedColormap
    """
    base = mpl.cm.get_cmap("coolwarm", N)           # original map, N samples
    colors = base(np.linspace(0, 1, N))             # RGBA array, shape (N,4)
    mid = N // 2                                    # neutral grey index
    lo_rgba = mpl.colors.to_rgba(low)

    # Replace the lower half [0 : mid] with a linear blend: `low` → mid-grey
    t = np.linspace(0, 1, mid + 1)[:, None]         # 0 → 1 from low to centre
    colors[:mid + 1] = (1 - t) * lo_rgba + t * colors[mid]

    return ListedColormap(colors, name=name)

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

vmin = -0.05
vmax = 0.05

#cmap = kde.plot.main.custom_diverging_cmap(low=ACR_BLUE, high=NNXR_GRAY)
cmap = modified_coolwarm_low(low=ACR_BLUE)

all_nnxo = [reb_rel_avgs[subject]['NNXo'] for subject in reb_rel_avgs.keys()]
all_nnxr = [reb_rel_avgs[subject]['NNXr'] for subject in reb_rel_avgs.keys()]
all_nnxo = acr.sync.average_sttc_matrices(all_nnxo)
all_nnxr = acr.sync.average_sttc_matrices(all_nnxr)

diffs = all_nnxo - all_nnxr
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
f, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.heatmap(diffs, cmap=cmap, ax=ax, norm=norm, cbar_kws={'ticks': [vmin, 0, vmax]})

fig_id = 'REBOUND_full_channel_map_all_subjects'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f.savefig(fig_path, transparent=True, dpi=600, bbox_inches='tight')

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

vmin = -0.05
vmax = 0.05

#cmap = kde.plot.main.custom_diverging_cmap(low=ACR_BLUE, high=NNXR_GRAY)
cmap = modified_coolwarm_low(low=ACR_BLUE)
all_nnxo = [reb_rel_avgs[subject]['NNXo'] for subject in reb_rel_avgs.keys()]
all_nnxr = [reb_rel_avgs[subject]['NNXr'] for subject in reb_rel_avgs.keys()]
all_nnxo = acr.sync.average_sttc_matrices(all_nnxo)
all_nnxr = acr.sync.average_sttc_matrices(all_nnxr)

diffs = all_nnxo - all_nnxr
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
f, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.heatmap(diffs, cmap=cmap, ax=ax, norm=norm, cbar=False)

fig_id = 'REBOUND_full_channel_map_all_subjects_NOSCALE'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f.savefig(fig_path, transparent=True, dpi=600, bbox_inches='tight')

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

subject = 'ACR_33'
probe = 'NNXr'

cmap = kde.plot.main.custom_diverging_cmap(low='firebrick', high='black')
norm = TwoSlopeNorm(vmin=0.5, vcenter=1, vmax=2)
dat = reb_rel_avgs[subject][probe]


f, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.heatmap(dat, cmap=scm.roma_r, ax=ax, norm=norm, cbar_kws={'ticks': [0.5, 1, 2]})
plt.savefig(f'{notebook_figure_root}/{subject}--{probe}--rebound_sttc_map.png', transparent=True, dpi=600)

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

subject = 'ACR_33'
probe = 'NNXo'

cmap = kde.plot.main.custom_diverging_cmap(low='firebrick', high='black')
norm = TwoSlopeNorm(vmin=0.5, vcenter=1, vmax=2)
dat = reb_rel_avgs[subject][probe]


f, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.heatmap(dat, cmap=scm.roma_r, ax=ax, norm=norm, cbar_kws={'ticks': [0.5, 1, 2]})
plt.savefig(f'{notebook_figure_root}/{subject}--{probe}--rebound_sttc_map.png', transparent=True, dpi=600)

# Matrix - Subject-by-Subject

In [ ]:
plt.rcdefaults()
plt.style.use('fivethirtyeight')
for subject in reb_rel_avgs.keys():
    f, ax = plt.subplots(1, 2, figsize=(10, 5))
    sns.heatmap(reb_rel_avgs[subject]['NNXr'], cmap='PiYG', ax=ax[0],
                vmin=-0.1, vmax=0.1
                )
    ax[0].set_title(f'{subject} | {np.nanmean(reb_rel_avgs[subject]["NNXr"]):.2f}')
    sns.heatmap(reb_rel_avgs[subject]['NNXo'], cmap='PiYG', ax=ax[1], 
                vmin=-0.1, vmax=0.1
                )
    ax[1].set_title(f'{subject} | {np.nanmean(reb_rel_avgs[subject]["NNXo"]):.2f}')

In [ ]:
plt.rcdefaults()
plt.style.use('fivethirtyeight')
for subject in reb_rel_avgs.keys():
    nnxr_dat = reb_rel_avgs[subject]['NNXr']
    nnxo_dat = reb_rel_avgs[subject]['NNXo']
    nnxr_avg = np.nanmean(nnxr_dat)
    nnxo_avg = np.nanmean(nnxo_dat)
    nnxo_avg_norm = nnxo_avg/nnxr_avg
    rel_dat = nnxo_dat-nnxr_dat
    f, ax = plt.subplots(1, 1, figsize=(5, 5))
    sns.heatmap(nnxo_avg_norm, cmap='PiYG', ax=ax, vmin=0, vmax=2.0)
    ax.set_title(f'{subject} | {nnxo_avg_norm:.2f}')
    plt.show()

# Circ BL

In [ ]:
# make the averaged condition-matrices all relative to the full baseline
circ_rel_avgs = {}
for subject in bmtx.keys():
    circ_rel_avgs[subject] = {}
    for probe in ['NNXr', 'NNXo']:
        full_bl_mtx = cond_avgs[subject]['full_bl'][probe]
        reb_mtx = cond_avgs[subject]['circ_bl'][probe]
        rel_mtx = reb_mtx / full_bl_mtx
        circ_rel_avgs[subject][probe] = rel_mtx

In [ ]:
# make the averaged condition-matrices all relative to the full baseline - FISHER-transformed, difference
circ_rel_avgs = {}
for subject in bmtx.keys():
    circ_rel_avgs[subject] = {}
    for probe in ['NNXr', 'NNXo']:
        full_bl_mtx = cond_avgs[subject]['full_bl'][probe]
        circ_mtx = cond_avgs[subject]['circ_bl'][probe]
        eps = 1e-6
        full_bl_mtx = np.arctanh(np.clip(full_bl_mtx, -1+eps, 1-eps))
        circ_mtx = np.arctanh(np.clip(circ_mtx, -1+eps, 1-eps))
        circ_mtx = circ_mtx - full_bl_mtx
        circ_rel_avgs[subject][probe] = circ_mtx

In [ ]:
#obvious outlier/artifactual pairs
reb_rel_avgs['ACR_18']['NNXo'][1, 2] = np.nan
reb_rel_avgs['ACR_18']['NNXr'][1, 2] = np.nan
reb_rel_avgs['ACR_18']['NNXo'][3, 4] = np.nan
reb_rel_avgs['ACR_18']['NNXr'][3, 4] = np.nan
reb_rel_avgs['ACR_18']['NNXo'][4, 5] = np.nan
reb_rel_avgs['ACR_18']['NNXr'][4, 5] = np.nan

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

vmin = -0.05
vmax = 0.05

#cmap = kde.plot.main.custom_diverging_cmap(low=ACR_BLUE, high=NNXR_GRAY)
cmap = modified_coolwarm_low(low=ACR_BLUE)

all_nnxo = [circ_rel_avgs[subject]['NNXo'] for subject in circ_rel_avgs.keys()]
all_nnxr = [circ_rel_avgs[subject]['NNXr'] for subject in circ_rel_avgs.keys()]
all_nnxo = acr.sync.average_sttc_matrices(all_nnxo)
all_nnxr = acr.sync.average_sttc_matrices(all_nnxr)

diffs = all_nnxo - all_nnxr
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
f, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.heatmap(diffs, cmap=cmap, ax=ax, norm=norm, cbar_kws={'ticks': [vmin, 0, vmax]})

fig_id = 'CIRC-BL_full_channel_map_all_subjects'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f.savefig(fig_path, transparent=True, dpi=600, bbox_inches='tight')

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

vmin = -0.05
vmax = 0.05

#cmap = kde.plot.main.custom_diverging_cmap(low=ACR_BLUE, high=NNXR_GRAY)
cmap = modified_coolwarm_low(low=ACR_BLUE)

all_nnxo = [circ_rel_avgs[subject]['NNXo'] for subject in circ_rel_avgs.keys()]
all_nnxr = [circ_rel_avgs[subject]['NNXr'] for subject in circ_rel_avgs.keys()]
all_nnxo = acr.sync.average_sttc_matrices(all_nnxo)
all_nnxr = acr.sync.average_sttc_matrices(all_nnxr)

diffs = all_nnxo - all_nnxr
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
f, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.heatmap(diffs, cmap=cmap, ax=ax, norm=norm, cbar=False)

fig_id = 'CIRC-BL_full_channel_map_all_subjects--NOSCALE'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f.savefig(fig_path, transparent=True, dpi=600, bbox_inches='tight')

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

subject = 'ACR_33'
probe = 'NNXr'

cmap = kde.plot.main.custom_diverging_cmap(low='firebrick', high='black')
norm = TwoSlopeNorm(vmin=0.5, vcenter=1, vmax=2)
dat = circ_rel_avgs[subject][probe]


f, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.heatmap(dat, cmap=scm.roma_r, ax=ax, norm=norm, cbar_kws={'ticks': [0.5, 1, 2]})
plt.savefig(f'{notebook_figure_root}/{subject}--{probe}--CIRC-BL_sttc_map.png', transparent=True, dpi=600)

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

subject = 'ACR_33'
probe = 'NNXo'

cmap = kde.plot.main.custom_diverging_cmap(low='firebrick', high='black')
norm = TwoSlopeNorm(vmin=0.5, vcenter=1, vmax=2)
dat = circ_rel_avgs[subject][probe]


f, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.heatmap(dat, cmap=scm.roma_r, ax=ax, norm=norm, cbar_kws={'ticks': [0.5, 1, 2]})
plt.savefig(f'{notebook_figure_root}/{subject}--{probe}--CIRC-BL_sttc_map.png', transparent=True, dpi=600)

In [ ]:
# better averaging method - first average each condition across bouts into a single matrix, then make each channel-pair value relative to its own channel-pair value from the baseline. Then average all of those channel-pairs into a single number.
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False

fig_id = 'CIRC-BL_STTC_all_channel_average'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxr = np.array([np.nanmean(circ_rel_avgs[subject]['NNXr']) for subject in circ_rel_avgs.keys()])
nnxo = np.array([np.nanmean(circ_rel_avgs[subject]['NNXo']) for subject in circ_rel_avgs.keys()])
nnxr = nnxr[~np.isnan(nnxr)]
nnxo = nnxo[~np.isnan(nnxo)]
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=(3.5, 4))
print(ax.get_ylim())
ax.set_yticks([-.02, -.01, 0])
ax.set_ylim(-0.02, 0.005)
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
#stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))

if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# BY DEPTH

In [ ]:
acr.plots.supl()
nbroot = f'{acr.utils.PAPER_FIGURE_ROOT}/synchrony/sttc/by_depth'

In [ ]:
# Convert STTC matrix to dataframe with channel pairs
def mtx_to_df(mtx):
    sttc_df_list = []
    for i in range(mtx.shape[0]):
        for j in range(mtx.shape[1]):
            if i != j and not np.isnan(mtx[i, j]):  # Skip diagonal and NaN values
                sttc_df_list.append({
                    'sttc_value': mtx[i, j],
                    'channel_1': i+1,
                    'channel_2': j+1
                })

    sttc_df = pl.DataFrame(sttc_df_list)
    return sttc_df

In [ ]:
dfs = []
for subject in reb_rel_avgs.keys():
    for probe in ['NNXo', 'NNXr']:
        avg_mtx = reb_rel_avgs[subject][probe]
        df = mtx_to_df(avg_mtx)
        df = df.with_columns(pl.lit(probe).alias('probe'))
        df = df.with_columns(pl.lit(subject).alias('subject'))
        dfs.append(df)

mxdf = pl.concat(dfs)

In [ ]:
mxdf = acr.info_pipeline.label_df_sub_infra(mxdf, chan_col='channel_1', label_col='label_1', join_on=['subject', 'probe', 'channel_1'])

In [ ]:
mxdf = acr.info_pipeline.label_df_sub_infra(mxdf, chan_col='channel_2', label_col='label_2', join_on=['subject', 'probe', 'channel_2'])

In [ ]:
# INFRA - INFRA
fig_id = f'STTC_by_depth__infra-infra'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

filtdf = mxdf.filter(pl.col('label_1') == 'infragranular').filter(pl.col('label_2') == 'infragranular')
filtavg = filtdf.group_by(['subject', 'probe']).agg(pl.mean('sttc_value')).sort('subject', 'probe')
nnxo = filtavg.filter(pl.col('probe') == 'NNXo')['sttc_value'].to_numpy()
nnxr = filtavg.filter(pl.col('probe') == 'NNXr')['sttc_value'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats


In [ ]:
# GRAN - GRAN
fig_id = f'STTC_by_depth__gran-gran'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

filtdf = mxdf.filter(pl.col('label_1') == 'granular').filter(pl.col('label_2') == 'granular')
filtavg = filtdf.group_by(['subject', 'probe']).agg(pl.mean('sttc_value')).sort('subject', 'probe')
nnxo = filtavg.filter(pl.col('probe') == 'NNXo')['sttc_value'].to_numpy()
nnxr = filtavg.filter(pl.col('probe') == 'NNXr')['sttc_value'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats


In [ ]:
# SUB - SUB
fig_id = f'STTC_by_depth__sub-sub'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

filtdf = mxdf.filter(pl.col('label_1') == 'subgranular').filter(pl.col('label_2') == 'subgranular')
filtavg = filtdf.group_by(['subject', 'probe']).agg(pl.mean('sttc_value')).sort('subject', 'probe')
nnxo = filtavg.filter(pl.col('probe') == 'NNXo')['sttc_value'].to_numpy()
nnxr = filtavg.filter(pl.col('probe') == 'NNXr')['sttc_value'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats


In [ ]:
# INFRA - GRAN
fig_id = f'STTC_by_depth__infra-gran'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

filtdf = mxdf.filter(pl.col('label_1') == 'infragranular').filter(pl.col('label_2') == 'granular')
filtavg = filtdf.group_by(['subject', 'probe']).agg(pl.mean('sttc_value')).sort('subject', 'probe')
nnxo = filtavg.filter(pl.col('probe') == 'NNXo')['sttc_value'].to_numpy()
nnxr = filtavg.filter(pl.col('probe') == 'NNXr')['sttc_value'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

#stats = pg.ttest(nnxr, nnxo, paired=True)
stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    #acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats


In [ ]:
# INFRA - SUB
fig_id = f'STTC_by_depth__infra-sub'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

filtdf = mxdf.filter(pl.col('label_1') == 'infragranular').filter(pl.col('label_2') == 'subgranular')
filtavg = filtdf.group_by(['subject', 'probe']).agg(pl.mean('sttc_value')).sort('subject', 'probe')
nnxo = filtavg.filter(pl.col('probe') == 'NNXo')['sttc_value'].to_numpy()
nnxr = filtavg.filter(pl.col('probe') == 'NNXr')['sttc_value'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
#stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    #acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats


In [ ]:
# gran - SUB
fig_id = f'STTC_by_depth__gran-sub'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

filtdf = mxdf.filter(pl.col('label_1') == 'granular').filter(pl.col('label_2') == 'subgranular')
filtavg = filtdf.group_by(['subject', 'probe']).agg(pl.mean('sttc_value')).sort('subject', 'probe')
nnxo = filtavg.filter(pl.col('probe') == 'NNXo')['sttc_value'].to_numpy()
nnxr = filtavg.filter(pl.col('probe') == 'NNXr')['sttc_value'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
#stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    #acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# Schematic

In [ ]:
subject = 'ACR_33'
m = muas[subject]

In [ ]:
oodf = dag.load_binned_off_df(subject, sub_swi_exps[subject][0])
oodf = acr.oo_utils.enhance_oodf(oodf, full_hyps[subject], hyp_dicts[subject])

In [ ]:
roo = oodf.cdn('rebound').prb('NNXo')

In [ ]:
off_ends = roo['end_datetime'].to_pandas()

In [ ]:
for i in np.arange(0, 100, 10):
    t1 = off_ends[i]
    t2 = t1 + pd.Timedelta('1s')
    m2p = m.prb('NNXo').ts(t1, t2)
    f, ax = kde.plot.main.atomic_raster(m2p, figsize=(18, 4))
    ax.hlines(-8.5, t1+pd.Timedelta('20ms'), t1+pd.Timedelta('220ms'), colors='red', linewidth=2)
    ax.hlines(-10.5, t1+pd.Timedelta('20ms'), t1+pd.Timedelta('220ms'), colors='red', linewidth=2)

In [ ]:
t1 = off_ends[79] + pd.Timedelta('150ms')
t2 = t1 + pd.Timedelta('1s')
m2p = m.prb('NNXo').ts(t1, t2)
f, ax = kde.plot.main.atomic_raster(m2p, figsize=(18, 4), s=70)
sel1 = t1+pd.Timedelta('535ms')
sel2 = sel1+pd.Timedelta('300ms')
ax.hlines(-8.5, sel1, sel2, colors='red', linewidth=2)
ax.hlines(-10.5, sel1, sel2, colors='red', linewidth=2)
#ax.set_ylim(-10.9, -8.1)
#x.set_xlim(t1+pd.Timedelta('20ms'), t1+pd.Timedelta('220ms'))

In [ ]:
t1 = off_ends[79] + pd.Timedelta('150ms')
t2 = t1 + pd.Timedelta('1s')
m2p = m.prb('NNXo').ts(t1, t2)
f, ax = kde.plot.main.atomic_raster(m2p, figsize=(18, 4), s=160)
sel1 = t1+pd.Timedelta('535ms')
sel2 = sel1+pd.Timedelta('300ms')
ax.hlines(-8.5, sel1, sel2, colors='red', linewidth=2)
ax.hlines(-10.5, sel1, sel2, colors='red', linewidth=2)
ax.set_ylim(-10.9, -8.1)
ax.set_xlim(sel1, sel2)
tm2p = m2p.ts(sel1, sel2)
for spk_time in tm2p.filter(pl.col('negchan')==-9)['datetime'].to_pandas():
    s1 = spk_time-pd.Timedelta('5ms')
    s2 = spk_time+pd.Timedelta('5ms')
    ax.hlines(-9, s1, s2, colors='green', linewidth=6, alpha=0.3)
    
for spk_time in tm2p.filter(pl.col('negchan')==-10)['datetime'].to_pandas():
    s1 = spk_time-pd.Timedelta('5ms')
    s2 = spk_time+pd.Timedelta('5ms')
    ax.hlines(-10, s1, s2, colors='green', linewidth=6, alpha=0.0)

In [ ]:
tm2p = m2p.ts(t1+pd.Timedelta('20ms'), t1+pd.Timedelta('220ms'))

In [ ]:
t1 = off_ends[10]
t2 = t1 + pd.Timedelta('2s')
m2p = m.prb('NNXo').ts(t1, t2)
f, ax = kde.plot.main.atomic_raster(m2p, figsize=(18, 4), s=120)
ax.hlines(-8.5, t1+pd.Timedelta('20ms'), t1+pd.Timedelta('220ms'), colors='red', linewidth=2)
ax.hlines(-10.5, t1+pd.Timedelta('20ms'), t1+pd.Timedelta('220ms'), colors='red', linewidth=2)
ax.set_ylim(-10.9, -8.1)
ax.set_xlim(t1+pd.Timedelta('20ms'), t1+pd.Timedelta('220ms'))

for spk_time in tm2p.filter(pl.col('negchan')==-9)['datetime'].to_pandas():
    s1 = spk_time-pd.Timedelta('5ms')
    s2 = spk_time+pd.Timedelta('5ms')
    ax.hlines(-9, s1, s2, colors='green', linewidth=6, alpha=0.3)
    
for spk_time in tm2p.filter(pl.col('negchan')==-10)['datetime'].to_pandas():
    s1 = spk_time-pd.Timedelta('5ms')
    s2 = spk_time+pd.Timedelta('5ms')
    ax.hlines(-10, s1, s2, colors='green', linewidth=6, alpha=0.3)

In [ ]:
for i in range(2):
    t1 = off_ends[i]-pd.Timedelta('20ms')
    t2 = t1 + pd.Timedelta('180ms')
    m2p = m.prb('NNXo').ts(t1, t2)
    f, ax = kde.plot.main.atomic_raster(m2p, figsize=(8, 4))
    ax.hlines(-8, t1+pd.Timedelta('20ms'), t1+pd.Timedelta('30ms'), colors='red', linewidth=2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------------------------------------------
# Parameters
# -------------------------------------------------------------------
K = 16                        # number of channels
radius = 1.0                  # radius for circle layout

# random demo matrix (upper‑triangular values only)
rng = np.random.default_rng(42)
mat = np.zeros((K, K))
iu = np.triu_indices(K, k=1)
mat[iu] = rng.random(size=iu[0].size)          # random weights for demo
mat = np.where(np.triu(np.ones_like(mat), 1) == 1, mat, np.nan)  # mask lower & diag

# -------------------------------------------------------------------
# Circle layout coordinates
# -------------------------------------------------------------------
angles = np.linspace(0, 2 * np.pi, K, endpoint=False)
coords = np.column_stack((radius * np.cos(angles),
                          radius * np.sin(angles)))

# -------------------------------------------------------------------
# Plot
# -------------------------------------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(30, 15))

# ---- left: circle of channels with all pairwise edges ----
ax = axes[0]
for i in range(K):
    xi, yi = coords[i]
    for j in range(i + 1, K):
        xj, yj = coords[j]
        ax.plot([xi, xj], [yi, yj], linewidth=0.5, alpha=0.3)

ax.scatter(coords[:, 0], coords[:, 1], s=60)
for idx, (x, y) in enumerate(coords):
    ax.text(x, y, f"{idx}", ha='center', va='center', fontsize=8,
            color='white', weight='bold')

ax.set_aspect('equal')
ax.axis('off')
ax.set_title("All 120 pairwise\nchannel comparisons")

# ---- right: STTC upper‑triangle heatmap ----
ax = axes[1]
im = ax.imshow(mat, interpolation='none')
ax.set_title("Matrix representation\n(upper triangle)")

# ticks and labels
ax.set_xticks(np.arange(K))
ax.set_yticks(np.arange(K))
ax.set_xticklabels(np.arange(K))
ax.set_yticklabels(np.arange(K))
ax.set_xlabel("Channel")
ax.set_ylabel("Channel")

# rotate x tick labels for clarity
plt.setp(ax.get_xticklabels(), rotation=90)

# colorbar
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04, label="STTC value")

plt.tight_layout()

In [ ]:
# -------------------------------------------------------------------
# Parameters
# -------------------------------------------------------------------
K = 16                        # number of channels
radius = 1.0                  # radius for circle layout

# random demo matrix (upper‑triangular values only)
rng = np.random.default_rng(42)
mat = np.zeros((K, K))
iu = np.triu_indices(K, k=1)
mat[iu] = rng.random(size=iu[0].size)          # random weights for demo
mat = np.where(np.triu(np.ones_like(mat), 1) == 1, mat, np.nan)  # mask lower & diag

# -------------------------------------------------------------------
# Circle layout coordinates
# -------------------------------------------------------------------
angles = np.linspace(0, 2 * np.pi, K, endpoint=False)
coords = np.column_stack((radius * np.cos(angles),
                          radius * np.sin(angles)))

# -------------------------------------------------------------------
# Plot
# -------------------------------------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(30, 15))

# ---- left: circle of channels with all pairwise edges ----
ax = axes[0]
for i in range(K):
    xi, yi = coords[i]
    for j in range(i + 1, K):
        xj, yj = coords[j]
        ax.plot([xi, xj], [yi, yj], linewidth=0.5, alpha=0.3)

ax.scatter(coords[:, 0], coords[:, 1], s=60)
for idx, (x, y) in enumerate(coords):
    ax.text(x, y, f"{idx}", ha='center', va='center', fontsize=8,
            color='white', weight='bold')

ax.set_aspect('equal')
ax.axis('off')
ax.set_title("All 120 pairwise\nchannel comparisons")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------------------------------------------
# Parameters
# -------------------------------------------------------------------
K = 16                 # number of channels on the linear probe
gap = 4              # horizontal gap between the two columns (arbitrary units)

# y‑coordinates for the linear probe (top = ch 0, bottom = ch 15)
y_coords = np.arange(K)[::-1]   # reverse so ch0 is at the top

# x‑coordinates for the two vertical stacks
x_left, x_right = 0.0, gap

# -------------------------------------------------------------------
# Create figure
# -------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(8, 18))

# ---- Draw every pairwise comparison as a line from left-channel i to right-channel j ----
for i in range(K - 1):
    yi = y_coords[i]
    for j in range(i + 1, K):
        yj = y_coords[j]
        ax.plot([x_left, x_right], [yi, yj], color=NNXR_GRAY, linewidth=3, alpha=0.6)

# ---- Draw the channel markers ----
ax.scatter(np.full(K, x_left),  y_coords, s=90, color='black', zorder=100)
ax.scatter(np.full(K, x_right), y_coords, s=90, color='black', zorder=100)

# ---- Aesthetics ----

ax.set_xlim(-0.2, gap + 0.2)
ax.set_ylim(-0.3, K+0.3)
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------------------------------------------
# Parameters
# -------------------------------------------------------------------
K = 16                 # number of channels on the linear probe
gap = 4              # horizontal gap between the two columns (arbitrary units)

# y‑coordinates for the linear probe (top = ch 0, bottom = ch 15)
y_coords = np.arange(K)[::-1]   # reverse so ch0 is at the top

# x‑coordinates for the two vertical stacks
x_left, x_right = 0.0, gap

# -------------------------------------------------------------------
# Create figure
# -------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(8, 12))

# ---- Draw every pairwise comparison as a line from left-channel i to right-channel j ----
for i in range(K - 1):
    yi = y_coords[i]
    for j in range(i + 1, K):
        yj = y_coords[j]
        ax.plot([x_left, x_right], [yi, yj], color=SOM_BLUE, linewidth=3, alpha=0.5)

# ---- Draw the channel markers ----
ax.scatter(np.full(K, x_left),  y_coords, s=140, color=SOM_BLUE, zorder=100)
ax.scatter(np.full(K, x_right), y_coords, s=140, color=SOM_BLUE, zorder=100)

# ---- Aesthetics ----

ax.set_xlim(-0.2, gap + 0.2)
#ax.set_ylim(-0.3, K+0.3)
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()
fig_name = 'channelwise_comparison_schematic'
plt.savefig(f'{notebook_figure_root}/{fig_name}.png', dpi=600, bbox_inches='tight', transparent=True)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------------------------------------------
# Parameters
# -------------------------------------------------------------------
K = 16                 # number of channels on the linear probe
gap = 4              # horizontal gap between the two columns (arbitrary units)

# y‑coordinates for the linear probe (top = ch 0, bottom = ch 15)
y_coords = np.arange(K)[::-1]   # reverse so ch0 is at the top

# x‑coordinates for the two vertical stacks
x_left, x_right = 0.0, gap

# -------------------------------------------------------------------
# Create figure
# -------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(8, 12))

# ---- Draw every pairwise comparison as a line from left-channel i to right-channel j ----
for i in range(K - 1):
    yi = y_coords[i]
    for j in range(i + 1, K):
        yj = y_coords[j]
        ax.plot([x_left, x_right], [yi, yj], color=NNXR_GRAY, linewidth=3, alpha=0.5)

# ---- Draw the channel markers ----
ax.scatter(np.full(K, x_left),  y_coords, s=140, color=NNXR_GRAY, zorder=100)
ax.scatter(np.full(K, x_right), y_coords, s=140, color=NNXR_GRAY, zorder=100)

# ---- Aesthetics ----

ax.set_xlim(-0.2, gap + 0.2)
#ax.set_ylim(-0.3, K+0.3)
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()
fig_name = 'channelwise_comparison_schematic--gray'
plt.savefig(f'{notebook_figure_root}/{fig_name}.png', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# -------------------------------------------------------------------
# Parameters
# -------------------------------------------------------------------
K = 16                        # number of channels
radius = 1.0                  # radius for circle layout

# random demo matrix (upper‑triangular values only)
rng = np.random.default_rng(42)
mat = np.zeros((K, K))
iu = np.triu_indices(K, k=1)
mat[iu] = rng.random(size=iu[0].size)          # random weights for demo
mat = np.where(np.triu(np.ones_like(mat), 1) == 1, mat, np.nan)  # mask lower & diag

# -------------------------------------------------------------------
# Circle layout coordinates
# -------------------------------------------------------------------
angles = np.linspace(0, 2 * np.pi, K, endpoint=False)
coords = np.column_stack((radius * np.cos(angles),
                          radius * np.sin(angles)))

# -------------------------------------------------------------------
# Plot
# -------------------------------------------------------------------
fig, axes = plt.subplots(1, 1, figsize=(15, 15))

# ---- left: circle of channels with all pairwise edges ----
ax = axes[0]
for i in range(K):
    xi, yi = coords[i]
    for j in range(i + 1, K):
        xj, yj = coords[j]
        ax.plot([xi, xj], [yi, yj], linewidth=0.5, alpha=0.3)

ax.scatter(coords[:, 0], coords[:, 1], s=60)
for idx, (x, y) in enumerate(coords):
    ax.text(x, y, f"{idx}", ha='center', va='center', fontsize=8,
            color='white', weight='bold')

ax.set_aspect('equal')
ax.axis('off')
ax.set_title("All 120 pairwise\nchannel comparisons")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch

# -------------------------------------------------------------------
# Parameters
# -------------------------------------------------------------------
K = 16                # number of channels on the linear probe
x0 = 0.0              # x‑coordinate of the probe
curv_unit = 0.7       # horizontal extent of the widest arc
line_w   = 3       # very thin lines
color    = "black"    # single solid colour
alpha    = 0.8        # no transparency  -> no shading build‑up

# y‑coordinates for the probe (top = ch 0, bottom = ch 15)
y_coords = np.arange(K)[::-1]            # ch0 at top

# -------------------------------------------------------------------
# Helper to draw a quadratic Bézier curve between two y positions
# -------------------------------------------------------------------
def make_arc(y_start, y_end, dx):
    """Return a PathPatch (quadratic curve) connecting (x0,y_start) → (x0,y_end)."""
    yc = 0.5 * (y_start + y_end)
    verts = [(x0, y_start),
             (x0 + dx, yc),        # control point
             (x0, y_end)]
    codes = [Path.MOVETO, Path.CURVE3, Path.CURVE3]
    return PathPatch(Path(verts, codes),
                     facecolor='none',   # absolutely no fill
                     edgecolor=color,
                     lw=line_w,
                     alpha=alpha)

# -------------------------------------------------------------------
# Create figure
# -------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(45, 30))

# ---- Alternate arcs left/right to reduce visual stacking ----
for i in range(K - 1):
    yi = y_coords[i]
    for j in range(i + 1, K):
        yj = y_coords[j]
        # Use distance to set horizontal deflection and side
        d   = j - i
        dx  = curv_unit * d / (K - 1)        # scale with distance
        dx  = dx if d % 2 == 0 else -dx       # alternate sides
        arc = make_arc(yi, yj, dx)
        ax.add_patch(arc)

# ---- Draw channel markers ----
ax.scatter(np.full(K, x0), y_coords, s=30, color='black', zorder=3)
for idx, y in enumerate(y_coords):
    ax.text(x0 - 0.25, y, f"{idx}", va='center', ha='right',
            fontsize=7, color='black')

# ---- Aesthetics ----
ax.set_title("Unique pairwise comparisons\nwithin one 16‑channel probe",
             fontsize=11)
ax.set_xlim(-curv_unit - 0.8, curv_unit + 0.8)
ax.set_ylim(-1, K)
#ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch

# -------------------------------------------------------------------
# Parameters
# -------------------------------------------------------------------
K = 6                # number of channels on the linear probe
x0 = 0.0              # x‑coordinate of the probe
curv_scale = 0.5      # controls horizontal extent of the arcs

# y‑coordinates for the probe (top = ch 0, bottom = ch 15)
y_coords = np.arange(K)[::-1]   # channel 0 at top

# -------------------------------------------------------------------
# Helper to draw a quadratic Bézier curve between two y positions
# -------------------------------------------------------------------
def make_arc(y_start, y_end, dx):
    """Return a PathPatch (quadratic curve) connecting (x0,y_start) → (x0,y_end)."""
    yc = 0.5 * (y_start + y_end)
    verts = [(x0, y_start),
             (x0 + dx, yc),        # control point (to the right)
             (x0, y_end)]
    codes = [Path.MOVETO, Path.CURVE3, Path.CURVE3]
    return PathPatch(Path(verts, codes), lw=.0001, alpha=0.1, color='blue')

# -------------------------------------------------------------------
# Create figure
# -------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(15, 30))

# ---- Draw arcs for every unique pair (i<j) ----
for i in range(K - 1):
    yi = y_coords[i]
    for j in range(i + 1, K):
        yj = y_coords[j]
        dx = curv_scale * (j - i)          # wider arc for farther pairs
        arc = make_arc(yi, yj, dx)
        ax.add_patch(arc)

# ---- Draw channel markers ----
ax.scatter(np.full(K, x0), y_coords, s=200, color='black')
for idx, y in enumerate(y_coords):
    ax.text(x0 - 0.3, y, f"{idx}", va='center', ha='right',
            fontsize=8, color='black')

# ---- Aesthetics ----
ax.set_title("All 120 pairwise comparisons\nwithin one 16‑channel probe",
             fontsize=12)
ax.set_xlim(-1.5, curv_scale * (K - 1) + 1.0)
ax.set_ylim(-1, K)
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------------------------------------------
# Parameters
# -------------------------------------------------------------------
K = 16                      # number of channels
cell_size = 1.0             # size of each matrix cell in arbitrary units
lw_line = 0.4               # line width
alpha_line = 0.6            # line transparency (keeps the web readable)

# -------------------------------------------------------------------
# Coordinates for grid cells
# -------------------------------------------------------------------
# Cell (row=i, col=j) will be drawn at:
# x from j to j+1  (left edge), y from (K-1-i) to (K-i) (top edge)
# (We flip y so that row 0 is at top)

fig, ax = plt.subplots(figsize=(20, 20))

# ---- Draw grid (optional: faint grid lines) ----
for k in range(K + 1):
    ax.plot([0, K], [k, k], color='lightgray', linewidth=0.5)  # horizontal
    ax.plot([k, k], [0, K], color='lightgray', linewidth=0.5)  # vertical

# ---- Draw lines from top-axis labels to upper‑triangle cells ----
for j in range(1, K):             # start from column 1 (col 0 has no upper cells)
    x_top = j + 0.5               # center of column j at top
    y_top = K + 0.2               # a bit above the grid
    for i in range(j):            # rows 0..j-1 (upper triangle)
        # center of the target cell
        x_cell = j + 0.5
        y_cell = K - (i + 0.5)
        ax.plot([x_top, x_cell], [y_top, y_cell],
                color='black', linewidth=lw_line, alpha=alpha_line)

# ---- Label axes ----
# Top axis labels (columns)
for j in range(K):
    ax.text(j + 0.5, K + 0.5, str(j), ha='center', va='center', fontsize=7)

# Right axis labels (rows)
for i in range(K):
    ax.text(K + 0.5, K - (i + 0.5), str(i), ha='center', va='center', fontsize=7)

# ---- Aesthetics ----
ax.set_xlim(0, K + 2)                  # extra space on right for labels
ax.set_ylim(0, K + 1)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title("Mapping each channel (top) to\nits pairwise comparison matrix cell",
             fontsize=10)

plt.tight_layout()
